RNN LSTM for text classification
=============
<span style="color: lightsteelblue;">Deep Learning</span>

The goal of this notebook is to train recurrent neural network with LSTM cell for the purpose of text classification.

## 1. Data preparation

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import collections
import numpy as np
import random
import re
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
def clean(string):
  string = re.sub(r"[^А-Яа-я0-9(),!?]", " ", string)
  string = re.sub(r",", " , ", string)
  string = re.sub(r"!", " ! ", string)
  string = re.sub(r"\(", " \( ", string)
  string = re.sub(r"\)", " \) ", string)
  string = re.sub(r"\?", " \? ", string)
  string = re.sub(r"\s{2,}", " ", string)
  return string.strip().lower()

def read_data(folder):
  data = {}
  data_files = os.listdir(folder)
  for data_file in data_files:
    with open(os.path.join(folder,data_file)) as f:
      articles = []
      for line in f:
        articles.append(line)
    data[data_file] = articles

  return data

data = read_data('articles')
num_classes = len(data)

total_articles_count = 0
labels = []
data_x = []
data_y = []

for label_index, label in enumerate(data):
  labels.append(label)

  for article in data[label]:
    sentences = re.split('[\.]', article)
    for sentence in sentences:
      words = clean(sentence).split(' ')

      data_x = data_x + words
      data_y.append(label_index)

data_size = len(data_x)

print('Total input size: %d words\n' % data_size)

for label in data:
  excerpt = data[label][0][:175] + '...'
  print('Number of %s articles is %d. For example: \n%s' % (label, len(data[label]), excerpt))

Total input size: 6508 words

Number of business articles is 3. For example: 
Индийский слон Кофейный рынок лихорадит. К плохим новостям из Южной Америки, гуляющим уже несколько недель, на днях добавились тревожные звонки из Индии, которая является веду...
Number of culture articles is 6. For example: 
Если погуглить рекомендательные подборки семейных саг, то на разных литературных сайтах в списке из условных 10 позиций непременно окажется «Сага о Форсайтах» и, надо думать, ...
Number of sport articles is 4. For example: 
Сами себе чемпионы Среди всех участников чемпионата мира на этот раз было лишь 19 обладателей паспорта в бордовой корочке. Хотя формально ни одного россиянина на турнире не бы...


In [8]:
vocabulary_size = 2600

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(data_x)

train_size = round(data_size * 0.95)
valid_size = data_size - train_size

train_data   = np.array(data[:train_size])
train_labels = np.array(data_y[:train_size])
valid_data   = np.array(data[train_size:])
valid_labels = np.array(data_y[train_size:])

print(count[:10])
print(dictionary['слон'])
print(reverse_dictionary[3])

[['UNK', 162], (',', 627), ('в', 227), ('и', 197), ('на', 110), ('не', 86), ('что', 67), ('с', 57), ('это', 55), ('как', 53)]
684
и


Function to generate a training batch for the LSTM model:

In [9]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]

    self._last_batch = self._next_batch()


  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, self._text[self._cursor[b]]] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch

  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def words(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  words back into its (most likely) word representation."""
  return [reverse_dictionary[c] for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  for b in batches:
    s = [word for word in words(b)]
  return s

train_batches = BatchGenerator(train_data, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_data, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['америки', 'в', '10', 'пойдут', 'общепита', '2016', 'сейчас', 'основания', 'компании', 'на', 'от', 'это', 'содержится', 'авиастроительной', 'го', 'и', 'в', 'материальных', 'являются', 'разделу', 'сухого', 'сага', 'форстер', 'ее', ',', 'материального', 'форсайтах', '', 'из', 'когда', 'цикл', 'небольшого', 'и', ',', 'с', 'хх', 'живете', 'еще', 'нет', 'и', 'тирана', 'ни', 'побороть', 'и', 'было', 'наконец', 'няньки', 'социальные', 'инвалиды', ',', ',', 'что', 'можем', 'их', 'зрителю', 'этой', 'мило', ',', 'шубенков', 'мария', ',', 'вот', 'стал', 'эстафете']
['тревожные', 'а', 'йорка', 'этой', 'подорожал', 'кофе', 'текущего', 'анонимности', 'экономическая', 'больше', 'что', 'конечно', 'чашке', 'самолете', 'новак', ',', 'самолетов', 'финансовом', 'от', ',', 'и', 'из', 'такого', ',', 'них', 'но', 'любовь', 'не', 'прежний', 'то', ',', 'ощущениям', 'долго', ',', ',', ',', 'городе', 'таким', 'разному', 'у', 'ирэн', 'что', 'любви', 'умеют', 'уэллера', 'предубеждение', ',', 'нее', 'сейчас', 'год

In [ ]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

LSTM Model:

In [ ]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))

  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)

    return output_gate * tf.tanh(state), state

  # Input data.
  train_inputs = tf.placeholder(tf.float32, shape=[None, vocabulary_size])
  train_labels = tf.placeholder(tf.float32)

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
      labels=train_labels,
      logits=logits
    ))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)

  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [41]:
alph = "abcdefghijklmnoprst"
chars = []

for _ in range(10):
  r_char = random.choice(list(alph))
  chars.append(r_char)

print(chars)
print(chars[:3])
print(chars[3:])

['m', 'g', 'r', 'c', 'c', 'a', 'o', 'o', 'r', 'k']
['m', 'g', 'r']
['c', 'c', 'a', 'o', 'o', 'r', 'k']
